# Amazon Knowledge Base Connection Test
Este notebook permite probar la conexión con Amazon Knowledge Bases utilizando las credenciales definidas en el archivo `.env`.

In [1]:
import os
import sys
import asyncio
import json
import boto3
from dotenv import load_dotenv

sys.path.append(os.getcwd())
load_dotenv()

# Mapear credenciales para boto3 si están en minúsculas en el .env
for key in ["aws_access_key_id", "aws_secret_access_key", "aws_region", "aws_session_token"]:
    val = os.getenv(key)
    if val:
        os.environ[key.upper()] = val

from src.config import settings
from src.infrastructure.aws_kb import AmazonKnowledgeBaseProvider
from src.core.models import QueryRequest

print(f"✅ Región configurada en Settings: {settings.AWS_REGION}")
print(f"✅ Perfil configurado: {settings.AWS_PROFILE}")

✅ Región configurada en Settings: us-east-1
✅ Perfil configurado: None


## 1. Verificar Identidad y Región Real
Vamos a ver quién somos y en qué región está operando boto3 por defecto.

In [2]:
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    session = boto3.Session()
    print(f"✅ Conectado como: {identity['Arn']}")
    print(f"✅ Región de la sesión: {session.region_name}")
except Exception as e:
    print(f"❌ Error de AWS: {e}")

✅ Conectado como: arn:aws:iam::846588355404:user/admin-user-05112025
✅ Región de la sesión: us-east-1


## 2. Listar KBs y obtener su Región
Buscamos la KB y extraemos su ARN para estar 100% seguros de su región.

In [3]:
provider = AmazonKnowledgeBaseProvider()
client = boto3.client('bedrock-agent', region_name=settings.AWS_REGION)

async def get_all_kbs_debug():
    print(f"Listando KBs en la región {settings.AWS_REGION}...")
    summaries = client.list_knowledge_bases().get('knowledgeBaseSummaries', [])
    
    if not summaries:
        print(f"⚠️ No se encontraron KBs en {settings.AWS_REGION}. ¿Es esta la región correcta?")
        return []

    results = []
    for kb in summaries:
        kb_id = kb['knowledgeBaseId']
        # Obtener detalles para ver el ARN
        details = client.get_knowledge_base(knowledgeBaseId=kb_id).get('knowledgeBase', {})
        arn = details.get('knowledgeBaseArn', 'N/A')
        print(f"- {kb['name']} | ID: {kb_id} | ARN: {arn}")
        results.append(kb_id)
    return results

all_ids = await get_all_kbs_debug()

Listando KBs en la región us-east-1...
- kb-demo01-bsaravia | ID: IPJ5LJA6LQ | ARN: arn:aws:bedrock:us-east-1:846588355404:knowledge-base/IPJ5LJA6LQ


## 3. Probar Consulta
Si el ID existe pero falla la consulta, compararemos la región del ARN con la región de `settings.AWS_REGION`.

In [4]:
async def test_query_safe(kb_id):
    print(f"\nProbando query en KB: {kb_id} usando región {settings.AWS_REGION}...")
    request = QueryRequest(query="test", knowledge_base_id=kb_id, num_results=1)
    try:
        resp = await provider.query(request)
        print("✅ ¡Éxito! Conexión completa establecida.")
    except Exception as e:
        print(f"❌ Falló la consulta: {e}")
        if "ResourceNotFoundException" in str(e):
            print("\n💡 ANÁLISIS: AWS dice que la KB no existe. Esto ocurre casi siempre porque")
            print(f"la KB está en una región distinta a '{settings.AWS_REGION}'.")
            print("Revisa el ARN arriba; si dice 'us-east-1' pero tu config dice 'us-west-2', deberás cambiar la región en el .env.")

if all_ids:
    await test_query_safe(all_ids[0])


Probando query en KB: IPJ5LJA6LQ usando región us-east-1...
✅ ¡Éxito! Conexión completa establecida.


In [4]:
from rich import print as rprint
request = QueryRequest(query="que informacion hay en esta base de conocimiento?", knowledge_base_id=all_ids[0], num_results=3)
res = await provider.query(request)
rprint(res)

QueryResponse(
    results=[
        SearchResult(
            content={
                'text': 'COLECCIONES DE PROMPTS     A continuación, se presentan algunas colecciones destacadas de 
prompts:     Biblioteca de prompts de Content Scale26: Esta biblioteca está diseñada para abordar una amplia gama 
de casos de uso y necesidades, dirigida a modelos de IA como ChatGPT, Bard, Midjourney, DALL-E 2, entre otros. 
Permite a los usuarios explorar categorías específicas para obtener respuestas detalladas o iniciar proyectos 
creativos de diversas índoles.     Prompts para la creación de contenido27: Esta colección incluye sugerencias bien
elaboradas por expertos como Amin Boulouma, Robert Brandl y Charlotte Evans. Abarca áreas como ventas y marketing, 
atención al cliente, desarrollo de productos, creación de contenido y redacción publicitaria, proporcionando 
herramientas útiles para mejorar la generación de contenidos.REPORTE DE TECNOLOGÍA: AI GENERATIVA 18     Prompts 
para arte generado por IA27: Para quienes buscan generar arte mediante IA, existen colecciones específicas de 
prompts para plataformas como DALL-E, Midjourney y Stable Diffusion.',
                'type': 'TEXT'
            },
            location={
                's3Location': {
                    'uri': 's3://s3-prueba-docs-08sept2025/docs_demo01/Reporte-de-tecnologia-IA-Generativa.pdf'
                },
                'type': 'S3'
            },
            score=0.6259970366954803
        ),
        SearchResult(
            content={
                'text': 'Aurora: Aurora es la plataforma interna de IA Generativa del Banco Interamericano de 
Desarrollo, diseñada para mejorar la eficiencia en la búsqueda, el resumen y la generación de contenido a partir 
deREPORTE DE TECNOLOGÍA: AI GENERATIVA 14     documentos institucionales. La plataforma alberga múltiples 
asistentes, cada uno de ellos diseñado para acceder a conjuntos específicos de documentos, lo que permite a los 
usuarios enfocar sus consultas y respuestas. Aurora ayuda a los usuarios a encontrar información rápidamente, 
resumir los puntos clave y generar nuevo contenido, todo ello dentro de un entorno seguro.     Seek: Disponible en 
el sitio web de publicaciones del Grupo BID, facilita y acelera el acceso y la utilización del conocimiento dentro 
de nuestras publicaciones, mediante el uso de IA Generativa1. Seek genera una respuesta basada exclusivamente en la
información contenida en la publicación. Además, la respuesta incluye una cita vinculada que permite a los usuarios
navegar directamente a la página relacionada con la respuesta para obtener más detalles.     ATAL: El BID está 
llevando a cabo pruebas con un chatbot de IA Generativa diseñado para facilitar material didáctico en el marco del 
programa «Aprendamos Todos a Leer» (ATAL).',
                'type': 'TEXT'
            },
            location={
                's3Location': {
                    'uri': 's3://s3-prueba-docs-08sept2025/docs_demo01/Reporte-de-tecnologia-IA-Generativa.pdf'
                },
                'type': 'S3'
            },
            score=0.6158619523048401
        ),
        SearchResult(
            content={
                'text': 'aprender y crear contenido nuevo que emula la creatividad humana.     Utilizando 
algoritmos sofisticados y grandes volúmenes de datos, la IA Generativa no solo está transformando la forma en que 
las máquinas interactúan con el mundo, sino también en cómo contribuyen a una gran variedad de sectores, desde la 
salud hasta la eficiencia en la productividad o la creatividad. La habilidad de estos sistemas para generar nuevos 
puntos de datos, textos o soluciones basadas en patrones aprendidos destaca su potencial para revolucionar 
industrias e impulsar la innovación. Dado su crecimiento y expansión, comprender las capacidades y las 
implicaciones de la IA Generativa se ha convertido en un aspecto clave tanto para individuos como para 
organizaciones.     Este informe tien